# Create a New Virtual Environment

We begin by creating a new virtual environment for our geocoding work. As a reminder, a virtual environment is a self-contained Python environment, generally built to accommodate a specific project. We use these so that we're not endlessly installing pacakges into a single bloated environment; instead, we work only with those packages we need. In a terminal window type...

```sh
# Tells python 3 to run the venv module...
# ...and creating a new virtual environment...
# ...in ~/.venv/geocode
python3 -m venv ~/.venv/geocode
```

We can activate this virtual environment by running the following at the terminal:

```sh

```

We begin by installing the `opencage` geocoder. 

```sh
pip install opencage
```

## Import Geocoder Modules

In [2]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [3]:
key = '563374d7ab7a48739e840a5dd3d5fa8b'
geocoder = OpenCageGeocode(key)

In [5]:
# Somerville Public Library
results = geocoder.reverse_geocode(42.385776, -71.094382)
pprint(results)

[{'annotations': {'DMS': {'lat': "42° 23' 8.83716'' N",
                          'lng': "71° 5' 39.85944'' W"},
                  'FIPS': {'county': '25017', 'state': '25'},
                  'MGRS': '19TCG2759494735',
                  'Maidenhead': 'FN42kj82qo',
                  'Mercator': {'x': -7914193.002, 'y': 5190132.398},
                  'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=29794889#map=16/42.38579/-71.09441',
                          'note_url': 'https://www.openstreetmap.org/note/new#map=16/42.38579/-71.09441&layers=N',
                          'url': 'https://www.openstreetmap.org/?mlat=42.38579&mlon=-71.09441#map=16/42.38579/-71.09441'},
                  'UN_M49': {'regions': {'AMERICAS': '019',
                                         'NORTHERN_AMERICA': '021',
                                         'US': '840',
                                         'WORLD': '001'},
                             'statistical_groupings': ['MEDC']},
       

In [6]:
try:
  results = geocoder.reverse_geocode(42.385776, -71.094382, language='en', no_annotations='1')
  if results and len(results):
    print(results[0]['formatted'])
    # 11 Rue Sauteyron, 33800 Bordeaux, Frankreich
except RateLimitExceededError as ex:
  print(ex)

Somerville Central Library, 79 Highland Avenue, Somerville, MA 02143, United States of America


In [23]:
query = u'79 Highland Avenue, Somerville, MA 02143'
results = geocoder.geocode(query)

pprint(results)

42.385788;-71.094405;us;America/New_York


In [38]:


df = pd.DataFrame(
    {
        'neigh': [results[0]['components']['neighbourhood']],
        'lat': [results[0]['geometry']['lat']],
        'lng': [results[0]['geometry']['lng']],
        'w3w': results[0]['annotations']['what3words']['words']
    
    }
)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))

gdf.to_file('somerville_public.json', driver="GeoJSON")

In [55]:
addressfile = 'som_lib.txt'

try: 
    with open(addressfile,'r') as f:
        df = pd.DataFrame()
        for line in f:
            # Removes leading and trailing characters.
            address = line.strip()
            result = geocoder.geocode(address)
            if result and len(result):
                row = pd.Series({
                    'address': address,
                    'neigh': result[0]['components']['neighbourhood'],
                    'lat': result[0]['geometry']['lat'],
                    'lng': result[0]['geometry']['lng'],
                    'w3w': result[0]['annotations']['what3words']['words']
                })
                df = df.append(row, ignore_index=True)
                print(row)
            else:
                sys.stderr.write("not found: %s\n" % address)
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))
        gdf.to_file('somerville_public.json', driver="GeoJSON")
except IOError:
    print('Error: File %s does not appear to exist.' % addressfile)
except RateLimitExceededError as ex:
    print(ex)

address    115 Broadway, Somerville, MA 02145
neigh                         East Somerville
lat                                   42.3879
lng                                  -71.0836
w3w                       being.tanks.analogy
dtype: object
address    40 College Ave, Somerville, MA 02144
neigh                              Davis Square
lat                                     42.3981
lng                                    -71.1217
w3w                          nets.engine.played
dtype: object
address    79 Highland Avenue, Somerville, MA 02143
neigh                                   Spring Hill
lat                                         42.3858
lng                                        -71.0944
w3w                              beans.anyway.valve
dtype: object


In [20]:
import pandas as pd
import geopandas as gpd
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

key = '563374d7ab7a48739e840a5dd3d5fa8b'
geocoder = OpenCageGeocode(key)

reader = pd.read_csv('~/Desktop/data/becc/investors_exchange.csv', header=0)[0:5]

def gc(address):
    try: 
        address = str(address['add'])
        print(address)
        result = geocoder.geocode(address, )
        if result and len(result):
            row = pd.Series({
                'lat': result[0]['geometry']['lat'],
                'lng': result[0]['geometry']['lng'],
                'w3w': result[0]['annotations']['what3words']['words']
            })
            df = df.append(row, ignore_index=True)
            print(row)
        else:
            sys.stderr.write("not found: %s\n" % address)
        return(row)
    except RateLimitExceededError as ex:
        print(ex)

# reader = reader.merge(reader.apply(lambda add: gc(add), axis=1), left_index=True, right_index=True)
reader = reader.apply(lambda add: gc(add), axis=1)





750 West Pender St, Vancouver, BC V6C 1G8
750 West Pender St, Vancouver, BC V6C 1G8


NameError: ("name 'RateLimitExceededError' is not defined", 'occurred at index 0')

In [47]:
import pandas as pd
import geopandas as gpd
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import sys

key = '563374d7ab7a48739e840a5dd3d5fa8b'
geocoder = OpenCageGeocode(key)

def gc(row):
    address = row['add'].strip()
    print('Geocoding ' + address + '...')
    c_code = row['country'].strip()
    result = geocoder.geocode(address, countrycode=c_code)
    if result and len(result):
        row = pd.Series({
            'lat': result[0]['geometry']['lat'],
            'lng': result[0]['geometry']['lng']
        })
    else:
        row = pd.Series({
            'lat': None,
            'lng': None
        })
    return(row)

reader = pd.read_csv('~/Desktop/data/becc/investors_exchange.csv', header=0, delimiter=';', encoding='utf-8')
reader = reader.merge(reader.apply(lambda add: gc(add), axis=1), left_index=True, right_index=True)
gdf = gpd.GeoDataFrame(reader, geometry=gpd.points_from_xy(reader.lng, reader.lat))
gdf.to_file('/Users/ehuntley/Desktop/data/becc/investors_exchange.json', driver="GeoJSON")

Geocoding 750 West Pender St, Vancouver, BC V6C 1G8...
Geocoding 750 West Pender St, Vancouver, BC V6C 1G8...
Geocoding 1111 Melville St, Vancouver, BC V6E 3V6...
Geocoding 65 Rodney Road, Cheltenham, GL50 1HX...
Geocoding 220 Bay St, Toronto, ON M5J 2W4...
Geocoding 520 Wickham St, Fortitude Valley, QLD 4006...
Geocoding 600-890 West Pender St, Vancouver, BC V6C 1J9...
Geocoding 409 Granville St, Vancouver, BC V6C 1T2...
Geocoding 145 King St East, Toronto, ON M5C 2Y7...
Geocoding 17618 Ave, Surrey, BC V3S 1L3...
Geocoding 7001 E. Belleview Ave, Denver, CO 80237...
Geocoding 181 Bay St, Toronto, ON M5J 2T3...
Geocoding 1177 West Hastings St, Vancouver, BC V6E 4T5...
Geocoding 555 Burrard St, Vancouver, BC V7X 1M8...
Geocoding 1333 Johnston St, Vancouver, BC V6H 3R9...
Geocoding 409 Granville St, Vancouver, BC V6C 1T2...
Geocoding 38 Duffy Place, St Johns, NL, A1B 4M5...
Geocoding 800 West Pender St, Vancouver, BC V6C 2V6...
Geocoding 1199 West Pender St, Vancouver, BC V6E 2R1...
Geoco

Geocoding 1090 West Georgia St, Vancouver, BC V6E 3V7...
Geocoding 1111 Melville St, Vancouver, BC V6E 3V6...
Geocoding 200 Burrard St, Vancouver BC V6C 3L6...
Geocoding 580 Hornby St, Vancouver, BC V6C 3B6...
Geocoding 837 W Hastings St, Vancouver, BC V6C 3N6...
Geocoding 688 W Hastings St, Vancouver, BC V6B 1P1...
Geocoding 350 Indiana St, Golden, CO 80401...
Geocoding 200 Burrard St, Vancouver, BC V6C 3L6...
Geocoding 150 King St W, Toronto, ON M5H 1J9...
Geocoding 1030 W Georgia St, Vancouver, BC V6E 2Y3...
Geocoding 38 Avenue Rd Toronto, ON M5R 2G2...
Geocoding 133 Richmond St W, Toronto, ON M5H 2L3...
Geocoding 570 Granville St, Vancouver, BC V6C 3P1...
Geocoding 69 Yonge St, Toronto, ON M5E 1K3...
Geocoding 171 Donald St, Winnipeg, MB R3C 1M4...
Geocoding 80 Richmond St W, Toronto, ON M5H 2A4...
Geocoding 999 W Hastings St, Vancouver, BC V6C 2W2...
Geocoding 401 Bay St, Toronto, ON M5H 2Y4...
Geocoding 345 Chemin Castors Rouyn-Noranda, QC J9Y 0E7...
Geocoding 3335 Yonge St, Toro

Geocoding 56 Temperance St, Toronto, ON M5H 3V5...
Geocoding 409 Granville St, Vancouver BC V6C 1T2...
Geocoding 1040 W Georgia St, Vancouver, BC V6E 4H1...
Geocoding 1055 W Hastings St, Vancouver, BC V6E 2E9...
Geocoding 120 Adelaide St W, Toronto, ON M5H 1T1...
Geocoding 65 Queen St W, Toronto, ON M5H 2M5...
Geocoding 1199 W Hastings St, Vancouver, BC V6E 3T5...
Geocoding 700 Ave Dallaire, Rouyn-Noranda, QC J9X 4V9...
Geocoding 105 King St E, Toronto ON M5C 1G6...
Geocoding 278 Bay St, Thunder Bay, ON P7B 1R8...
Geocoding 145 Wellington St W, Toronto, ON M5J 1H8...
Geocoding 409 Granville St, Vancouver, BC V6C 1T2...
Geocoding 5 Canada Square, London, UK E14 5AQ...
Geocoding 1177 W Hastings St, Vancouver, BC V6E 2K3...
Geocoding 4750 Longley Ln, Reno, NV 89502...
Geocoding 1099 Kingston Rd, Pickering, ON L1V 1B5...
Geocoding 25 York St, Toronto ON MSJ 2V5...
Geocoding 1111 St Charles St W, Longueuil, QC J4K 5G4...
Geocoding 543 Granville St, Vancouver, BC V6C 1X8...
Geocoding 145 Kin

In [14]:
from geopy.geocoders import ArcGIS, Bing, Nominatim, OpenCage, GoogleV3, OpenMapQuest
import csv, sys
import pandas as pd
import keys

in_file = str(sys.argv[1])
out_file = str('gc_' + in_file)
timeout = int(sys.argv[2])

print('creating geocoding objects.')

arcgis = ArcGIS(timeout=timeout)
bing = Bing(api_key=keys.bing_api,timeout=100)
nominatim = Nominatim(user_agent=keys.n_user, timeout=timeout)
opencage = OpenCage(api_key=keys.oc_api,timeout=timeout)
googlev3 = GoogleV3(api_key=keys.g3_api, domain='maps.googleapis.com', timeout=timeout)
openmapquest = OpenMapQuest(api_key=keys.omq_api, timeout=timeout)

# choose and order your preference for geocoders here
geocoders = [openmapquest, nominatim, opencage, googlev3, arcgis]

def gc(address):
    street = str(address['street'])
    city = str(address['city'])
    state = str(address['state'])
    country = str(address['country'])
    add_concat = street + ", " + city + ", " + state + " " + country
    for gcoder in geocoders:
        location = gcoder.geocode(add_concat)
        if location != None:
            print(f'geocoded record {address.name}: {street}')
            located = pd.Series({
                'lat': location.latitude,
                'lng': location.longitude,
                'time': pd.to_datetime('now')
            })
        else:
            print(f'failed to geolocate record {address.name}: {street}')
            located = pd.Series({
                'lat': 'null',
                'lng': 'null',
                'time': pd.to_datetime('now')
            })
        return located

print('opening input.')
reader = pd.read_csv(in_file, header=0)
print('geocoding addresses.')
reader = reader.merge(reader.apply(lambda add: gc(add), axis=1), left_index=True, right_index=True)
print(f'writing to {out_file}.')
reader.to_csv(out_file, encoding='utf-8', index=False)
print('done.')


ModuleNotFoundError: No module named 'geopy'